# Segmenting and Clustering Neighborhoods in Toronto

## Import Library

In [ ]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

# library for pulling data out of HTML and XML files
from bs4 import BeautifulSoup

print('Libraries imported.')

In [ ]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
data_xml = requests.get(url).text

## Extract data from xml file with BeautifulSoup

In [ ]:
soup_postal_codes = BeautifulSoup(data_xml,'lxml')

# find the table in xml code
Table_postal_codes = soup_postal_codes.find('table',{'class':'wikitable sortable'})
Table_postal_codes = Table_postal_codes.find_all('tr')

In [ ]:
#loading empty array for board members
Post_code = []

for balise_tr in Table_postal_codes:
    balise_td = balise_tr.find_all('td')
    row = [i.text for i in balise_td]
    Post_code.append(row)
    #print(row)

del Post_code[0]
#Post_code

## Build the table of postal codes and to transform the data into a pandas dataframe :

In [ ]:
# instantiate the dataframe

column_names = ['PostalCode', 'Borough', 'Neighborhood'] 
Poste_code_table = pd.DataFrame(columns=column_names)

#Poste_code_table

In [ ]:
# Fill Poste_code_table with data from Post_code

for data in Post_code:
    
    Post_Code = data[0]
    Borough = data[1]
    Neighborhood = data[2]
    
    Poste_code_table = Poste_code_table.append({'PostalCode': Post_Code,
                                       'Borough': Borough,
                                       'Neighborhood':Neighborhood}, 
                                        ignore_index=True)

Poste_code_table.Neighborhood = Poste_code_table.Neighborhood.apply(lambda ele : ele.strip('\n')) 

In [ ]:
# Ignore cells with a borough that is Not assigned.
Poste_code_table = Poste_code_table[Poste_code_table['Borough'] != 'Not assigned']


In [ ]:
#If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. 

Poste_code_table = Poste_code_table.replace({'Neighborhood': r'Not assigned'}, 
                                            {'Neighborhood': Poste_code_table[Poste_code_table['Neighborhood'] == 'Not assigned'].Borough}, 
                                            regex=True)
                 


In [ ]:
# Combined into one row with the neighborhoods separated with a comma 

Poste_code_table = Poste_code_table.groupby(['PostalCode','Borough'],as_index=False).agg(lambda x : ', '.join(x))


In [ ]:
Poste_code_table.shape

In [ ]:
Poste_code_table